In [ ]:
# بارگذاری کتابخانه‌ها
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# اضافه کردن مسیر src
sys.path.append('../src')

# بارگذاری ماژول‌های پروژه
from src.data_generation.generators import BankingDataGenerator, NewUserGenerator
from src.data_generation.distributions import StatisticalDistributions
from src.data_generation.noise_injection import NoiseInjector
from src.database.sqlite_manager import SQLiteManager
from src.utils.config import get_config

# تنظیمات نمودار
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12
sns.set_style("whitegrid")

print("✅ تمام کتابخانه‌ها با موفقیت بارگذاری شدند")
print(f"📅 زمان اجرا: {datetime.now()}")


In [ ]:
# تولید داده نمونه
print("🔄 شروع تولید داده نمونه...")

# ایجاد دیتابیس نمونه
db_manager = SQLiteManager("../database/notebook_sample.db")

# ایجاد generator
generator = BankingDataGenerator(db_manager)

# تولید 100 کاربر نمونه
sample_stats = generator.generate_sample_data(num_users=100)

print("✅ تولید داده نمونه تکمیل شد!")
print(f"👥 کاربران تولیدشده: {sample_stats['total_users_generated']:,}")
print(f"💳 تراکنش‌های تولیدشده: {sample_stats['total_transactions_generated']:,}")
print(f"⏱️  زمان پردازش: {sample_stats['total_processing_time_seconds']:.2f} ثانیه")

# نمایش آمار دیتابیس
db_stats = db_manager.get_database_stats()
print(f"\n📊 آمار دیتابیس:")
print(f"   میانگین تراکنش در کاربر: {db_stats['avg_transactions_per_user']:.1f}")
print(f"   مجموع مبلغ تراکنش‌ها: {db_stats['total_amount']:,.0f} تومان")
print(f"   درصد نویز: {db_stats['noise_percentage']:.2f}%")


In [ ]:
# بارگذاری داده‌ها برای تحلیل
users_df = db_manager.execute_query("SELECT * FROM users LIMIT 100")
transactions_df = db_manager.execute_query("SELECT * FROM transactions LIMIT 1000")

print(f"📊 تعداد کاربران: {len(users_df)}")
print(f"📊 تعداد تراکنش‌ها: {len(transactions_df)}")

# نمایش چند رکورد نمونه
print("\n👥 نمونه کاربران:")
print(users_df.head())

print("\n💳 نمونه تراکنش‌ها:")
print(transactions_df.head())


In [ ]:
# نمودارهای توزیع داده‌ها
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# توزیع سن
axes[0, 0].hist(users_df['age'], bins=20, alpha=0.7, color='skyblue', edgecolor='black')
axes[0, 0].set_title('توزیع سن کاربران')
axes[0, 0].set_xlabel('سن')
axes[0, 0].set_ylabel('فراوانی')

# توزیع مبلغ تراکنش (لگاریتمی)
axes[0, 1].hist(np.log10(transactions_df['amount']), bins=30, alpha=0.7, color='lightgreen', edgecolor='black')
axes[0, 1].set_title('توزیع مبلغ تراکنش (لگاریتم)')
axes[0, 1].set_xlabel('لگاریتم مبلغ (تومان)')
axes[0, 1].set_ylabel('فراوانی')

# توزیع ساعت تراکنش
axes[0, 2].hist(transactions_df['hour_of_day'], bins=24, alpha=0.7, color='salmon', edgecolor='black')
axes[0, 2].set_title('توزیع ساعت تراکنش')
axes[0, 2].set_xlabel('ساعت')
axes[0, 2].set_ylabel('فراوانی')

# توزیع استان‌ها
province_counts = users_df['province'].value_counts().head(10)
axes[1, 0].bar(range(len(province_counts)), province_counts.values, color='orange', alpha=0.7)
axes[1, 0].set_title('توزیع استان‌ها (10 استان اول)')
axes[1, 0].set_xlabel('استان')
axes[1, 0].set_ylabel('تعداد کاربر')
axes[1, 0].tick_params(axis='x', rotation=45)

# توزیع انواع کارت
card_counts = users_df['preferred_card_type'].value_counts()
axes[1, 1].pie(card_counts.values, labels=card_counts.index, autopct='%1.1f%%', startangle=90)
axes[1, 1].set_title('توزیع انواع کارت')

# توزیع نویز
noise_counts = transactions_df['is_noise'].value_counts()
axes[1, 2].pie(noise_counts.values, labels=['عادی', 'نویز'], autopct='%1.1f%%', 
               colors=['lightblue', 'red'], startangle=90)
axes[1, 2].set_title('نسبت نویز در داده‌ها')

plt.tight_layout()
plt.show()

print("📈 نمودارهای توزیع داده‌ها نمایش داده شد")
print(f"🔍 درصد نویز: {(transactions_df['is_noise'].sum() / len(transactions_df)) * 100:.2f}%")


In [ ]:
# خلاصه نهایی
print("=== خلاصه تولید داده ===")
print(f"✅ {sample_stats['total_users_generated']:,} کاربر تولید شد")
print(f"✅ {sample_stats['total_transactions_generated']:,} تراکنش تولید شد") 
print(f"✅ نویز {db_stats['noise_percentage']:.2f}% اعمال شد")
print(f"✅ زمان پردازش: {sample_stats['total_processing_time_seconds']:.2f} ثانیه")

# نمایش توزیع‌های استفاده‌شده
dist_summary = generator.distributions.get_distribution_summary()
print(f"\n📊 توزیع‌های آماری استفاده‌شده:")
for dist_name, info in dist_summary.items():
    print(f"   {dist_name}: {info['type']}")

# آمار نویز
noise_stats = generator.noise_injector.get_noise_statistics()
print(f"\n🔍 آمار نویز:")
for noise_type, count in noise_stats['noise_counts'].items():
    if noise_type != 'total_noise_records' and count > 0:
        print(f"   {noise_type}: {count}")

print(f"\n🎯 سیستم آماده برای تولید dataset کامل!")
print(f"💡 برای تولید کامل اجرا کنید: python ../main.py --full")

# پاکسازی منابع
generator.close()
print(f"\n🧹 منابع پاک شدند")
